In [1]:
# @title
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import os
import sys
from datetime import datetime

# Definir o nome do arquivo a ser tratado
# IMPORTANTE: Inserir o nome do arquivo
nome_do_arquivo = 'ArquivoBruto_ControlePos_11032024'

parametroData = nome_do_arquivo.split("_")[2]
parametroOperation = nome_do_arquivo.split("_")[1]

nome_do_arquivo_baseVendas = f'BaseVendas_{parametroOperation}_{parametroData}'

caminho_do_arquivo = f'/content/drive/MyDrive/TIMrotinasPython/RetornoBruto/{nome_do_arquivo}.csv'
caminho_da_tabela_comparativa_URACheckout = '/content/drive/MyDrive/TIMPythonTabelasComparativa/tabelaComparativaURACheckout.csv'
caminho_da_base_vendas = f'/content/drive/MyDrive/TIMrotinasPython/BasesVendas/{nome_do_arquivo_baseVendas}.csv'


try:

    tabelaBruta = pd.read_csv(caminho_do_arquivo, sep=';', low_memory=False, on_bad_lines='skip')
    print(f"O arquivo: {nome_do_arquivo}.csv foi importado com sucesso!")
    tabelaBaseVendas = pd.read_csv(caminho_da_base_vendas, sep=';', low_memory=False, encoding='latin-1')
    print(f"O arquivo: {nome_do_arquivo_baseVendas}.csv foi importado com sucesso!")
    tabelaCompatativaURA = pd.read_csv(caminho_da_tabela_comparativa_URACheckout, sep=';', low_memory=False)

    padroes = ['1|', '2|', '3|', '9|']

    tabelaTratada = tabelaBruta.dropna(subset=['complemento'])
    tabelaTratada = tabelaTratada[tabelaTratada['complemento'].str.startswith(tuple(padroes))]
    tabelaTratada['complemento'] = tabelaTratada['complemento'].str.split('|').str[0]
    tabelaTratada['complemento'] = tabelaTratada['complemento'].astype(int)
    tabelaTratada['status_ura'] = tabelaTratada['complemento'].map(tabelaCompatativaURA.set_index('id_ura')['motivo'])
    tabelaTratada['data'] = pd.to_datetime(tabelaTratada["gravacao"], dayfirst=True).dt.date
    tabelaTratada = tabelaTratada.sort_values(by='complemento')
    tabelaTratada = tabelaTratada.drop_duplicates(subset='pessoa_id')

    tabelaTratada = tabelaTratada[[
                                  "historico_id"
                                , "gravacao"
                                , "data"
                                , "historico_tipo"
                                , "historico_tipo_descricao"
                                , "pessoa_id"
                                , "pessoa_nome"
                                , "ddd"
                                , "telefone"
                                , "usuario_gravou_codigo"
                                , "usuario_gravou_codigo_descricao"
                                , "complemento"
                                , "status_ura"
                                ]]
    print(f"Etapa de tratamento do arquivo: {nome_do_arquivo}.csv foi concluído com sucesso. Aguarde próxima etapa.")

    tabelaBaseVendas['DigitoURA'] = tabelaBaseVendas['PessoaID'].map(tabelaTratada.set_index('pessoa_id')['complemento'])
    tabelaBaseVendas['DigitoURA'] = tabelaBaseVendas['DigitoURA'].fillna(0)
    tabelaBaseVendas['DigitoURA'] = tabelaBaseVendas['DigitoURA'].astype(int)
    tabelaBaseVendas['StatusURACheckout'] = tabelaBaseVendas['PessoaID'].map(tabelaTratada.set_index('pessoa_id')['status_ura'])
    tabelaBaseVendas['StatusURACheckout'] = tabelaBaseVendas['StatusURACheckout'].fillna('Sem transferencia')
    tabelaBaseVendas['VENDA_ID'] = tabelaBaseVendas['PessoaID']
    tabelaBaseVendas['CPF_CLIENTE'] = tabelaBaseVendas['PessoaID']
    tabelaBaseVendas['CLIENTE'] = tabelaBaseVendas['PessoaID']

    print(f"Etapa de tratamento do arquivo: {nome_do_arquivo_baseVendas}.csv foi concluído com sucesso. Aguarde próxima etapa.")

    baseRelatorioFinal = tabelaBaseVendas[[
                          'VENDA_ID',
                          'DataHora',
                          'Matricula',
                          'Operador',
                          'CodigoVenda',
                          'TabVenda',
                          'CPF_CLIENTE',
                          'CLIENTE',
                          'NumTelefone',
                          'DataVenda',
                          'DigitoURA',
                          'StatusURACheckout',
                          'Matricula',
                          'Supervisor',
                          'FlegDeEnvioURA'
                        ]]

    baseRelatorioFinal = baseRelatorioFinal.rename(columns={
                                                          'VENDA_ID' : 'VENDA_ID',
                                                          'DataHora' : 'HORA_VENDA',
                                                          'Matricula' : 'USUARIO',
                                                          'Operador' : 'OPERADOR',
                                                          'CodigoVenda' : 'CODIGO_STATUS',
                                                          'TabVenda' : 'STATUS',
                                                          'CPF_CLIENTE' : 'CPF_CLIENTE',
                                                          'CLIENTE' : 'CLIENTE',
                                                          'NumTelefone' : 'DDDTEL',
                                                          'DataVenda' : 'DATA_VENDA',
                                                          'DigitoURA' : 'URACHECKOUT',
                                                          'StatusURACheckout' : 'STATUSCHECKOUT',
                                                          'Matricula' : 'MATRICULA',
                                                          'Supervisor' : 'SUPERVISOR',
                                                          'FlegDeEnvioURA' : 'FLEG_TRASNFER'
                                                          })

    print(f"Iniciando montagem de arquivo final")

    # Diretório onde deseja salvar o arquivo
    diretorio_salvar = '/content/drive/MyDrive/TIMDescargaDeURACheckout'

    nomeDoArquivoFinal = f'BaseURACheckout_{parametroOperation}_{parametroData}.csv'

    # Caminho completo para salvar o arquivo
    caminho_arquivo = os.path.join(diretorio_salvar, nomeDoArquivoFinal)

    # Salvar a tabela em formato CSV
    baseRelatorioFinal.to_csv(caminho_arquivo, encoding='latin-1', index=False)

    print(f"Concluído processo e arquivo: {nomeDoArquivoFinal} salvo!!")
    print(f"Diretório de armazenamento do arquivo: {diretorio_salvar}")
    print("================================================================================================")


except FileNotFoundError as e:
    # Captura a exceção e imprime qual arquivo não foi encontrado
    print(f"Arquivo não encontrado: {e}")
    print("Devido ao erro acima, o algoritmo foi encerrado")




Mounted at /content/drive
O arquivo: ArquivoBruto_ControlePos_11032024.csv foi importado com sucesso!
Arquivo não encontrado: [Errno 2] No such file or directory: '/content/drive/MyDrive/TIMrotinasPython/BasesVendas/BaseVendas_ControlePos_11032024.csv'
Devido ao erro acima, o algoritmo foi encerrado
